In [16]:
# initialize findspark for local
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.0.1/libexec")

In [12]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [19]:
ls

README.md       data/           dl.cfg          etl.py          get_data.ipynb


In [23]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [35]:
# store s3 bucket location
input_data = "s3://udacity-dend/song_data"

In [36]:
song_data = input_data + "/*/*/*/*.json"

# Get data

## create spark session

In [31]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [38]:
song_data = "s3a://udacity-dend/song_data/*/*/*/*.json"

## Get data from local

In [39]:
# read song data file
df = spark.read.json(song_data)

Py4JJavaError: An error occurred while calling o42.json.
: java.lang.NumberFormatException: For input string: "100M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1538)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSourc

In [ ]:
# extract columns to create songs table
songs_table = 

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table

In [ ]:
# extract columns to create artists table
artists_table = 

In [ ]:
# write artists table to parquet files
artists_table